## Scraping Detik

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time

In [2]:
# Mengecek Link status Ok
url = "https://news.detik.com/index"
response = requests.get(url)
response

<Response [200]>

In [3]:
def scrape_links(page_number):
    url = f"https://sport.detik.com/indeks/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article', {"class": "list-content__item"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
jumlah_index = 50
threads_link = []
links = []
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


Scraped 20 links from page 5
Scraped 20 links from page 19
Scraped 20 links from page 24
Scraped 20 links from page 16
Scraped 20 links from page 4
Scraped 20 links from page 40
Scraped 20 links from page 34
Scraped 20 links from page 14
Scraped 20 links from page 37
Scraped 20 links from page 33
Scraped 20 links from page 23
Scraped 20 links from page 44
Scraped 20 links from page 25
Scraped 20 links from page 27
Scraped 20 links from page 1
Scraped 20 links from page 18
Scraped 20 links from page 7
Scraped 20 links from page 13
Scraped 20 links from page 39
Scraped 20 links from page 12
Scraped 20 links from page 22
Scraped 20 links from page 11
Scraped 20 links from page 35
Scraped 20 links from page 8
Scraped 20 links from page 46
Scraped 20 links from page 30
Scraped 20 links from page 32
Scraped 20 links from page 3
Scraped 20 links from page 31
Scraped 20 links from page 17
Scraped 20 links from page 20
Scraped 20 links from page 41
Scraped 20 links from page 50
Scraped 20 links

In [5]:
results = []
# def scrape_url(url):
#     response = requests.get(url)
#     if response.status_code == 200:
#         # title =[]
#         # author = []
#         # date = []
#         # content = []
#         # category =[]
#         soup = BeautifulSoup(response.text, 'html.parser')
        
        
#         # Judul Berita
#         title_elem = soup.find('h1', {"class": "detail__title"})
#         if title_elem:
#             title_text = title_elem.text.strip()
#         else:
#             title_text = "Title not found"
#         # Author berita
#         author_elem = soup.find('div', {"class": "detail__author"})
#         if author_elem:
#             author_text = author_elem.get_text()
#             author_text = author_text.split('-')[0].strip()
#         else:
#             author_text = "Author not found"     
#         # tanggal berita
#         date_elem = soup.find('div', {"class": "detail__date"})
#         if date_elem:
#             date_text = date_elem.text.strip()
#         else:
#             date_text = "Date not found"
#         #     # Category berita
#         category_elements = soup.find('div',{"class": "page__breadcrumb"})
#         if category_elements:
#             category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
#             category_text= category_text.text.strip()
#         else:
#             category_text = "Category not found"
#         #     # Content Berita
#         body_elem = soup.find('div', {"class": "detail__body"})
        
#         if body_elem:
#             content_elem = body_elem.find_all('p')
#             content_text = ""
#             for p in content_elem:
#                 content_text += p.text.strip() + "\n"
            
#             if content_text.strip():
#                 content_text=content_text
#             else:
#                 content_text ="Content not found"
#         else:
#           content_text ="Content not found"

#         results.append({'judul_berita': title_text,
#                         'penulis' : author_text,
#                         'kategori_berita':category_text,
#                         'tanggal_berita': date_text,
#                         'isi_berita' : content_text,
#                         'link_berita' : url})
#     else:
#         print(f"Failed to retrieve data from {url}")

In [6]:
def scrape_url(url):
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # Extract data from the web page
                    title_elem = soup.find('h1', {"class": "detail__title"})
                    title_text = title_elem.text.strip() if title_elem else "Title not found"

                    author_elem = soup.find('div', {"class": "detail__author"})
                    author_text = author_elem.get_text().split('-')[0].strip() if author_elem else "Author not found"

                    date_elem = soup.find('div', {"class": "detail__date"})
                    date_text = date_elem.text.strip() if date_elem else "Date not found"

                    category_elements = soup.find('div', {"class": "page__breadcrumb"})
                    category_text = category_elements.find('a', {"dtr-sec": "breadcrumbkanal"}).text.strip() if category_elements else "Category not found"

                    body_elem = soup.find('div', {"class": "detail__body"})
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = "\n".join(p.text.strip() for p in content_elem)
                        content_text = content_text.replace('\n', ' ')
                    else:
                        content_text = "Content not found"

                    nama_berita_match = re.search(r'https://(?:www\.)?([a-zA-Z0-9.-]+)\.com', url)
                    nama_berita = nama_berita_match.group(1) if nama_berita_match else "Nama_berita not found"

                    results.append({'judul_berita': title_text,
                                            'penulis' : author_text,
                                            'kategori_berita':category_text,
                                            'tanggal_berita': date_text,
                                            'isi_berita' : content_text,
                                            'link_berita' : url})
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")        

In [7]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [ ]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 3000


,judul_berita,penulis,kategori_berita,tanggal_berita,isi_berita,link_berita
0,Marc Marquez Gagal Finis Lagi di Mandalika,Author not found,Category not found,"20,017 Views | Minggu, 15 Okt 2023 16:43 WIB",Marc Marquez kembali gagal finis di MotoGP Man...,https://20.detik.com/detikupdate/20231015-2310...
1,Siap Rebut Emas! Ini Daftar Pebulutangkis RI N...,Mercy Raya,Raket,"Rabu, 27 Sep 2023 14:10 WIB",PP PBSI merilis skuad Indonesia yang akan berl...,https://sport.detik.com/raket/d-6953098/siap-r...
2,Pemerintah Buka Peluang Sirkuit Mandalika Jadi...,Author not found,Category not found,"66,517 Views | Selasa, 17 Okt 2023 20:20 WIB",Pemerintah tidak menutup kemungkinan sirkuit M...,https://20.detik.com/detikupdate/20231017-2310...
3,Kata Marc Marquez Setelah Lagi-lagi Gagal Fini...,Author not found,Category not found,"13,750 Views | Senin, 16 Okt 2023 06:18 WIB","Marc Marquez gagal finis di MotoGP Mandalika, ...",https://20.detik.com/detikupdate/20231016-2310...
4,Basuki Hadimuljono Kasih Bonus Tambahan ke Per...,Mercy Raya,Sport Lain,"Rabu, 27 Sep 2023 10:30 WIB",Chef de Mission Basuki Hadimuljono gembira atl...,https://sport.detik.com/sport-lain/d-6952629/b...
5,Harapan Christian Hadinata buat Fajar/Rian: Ma...,Mercy Raya,Raket,"Kamis, 12 Okt 2023 14:55 WIB",Performa ganda putra Indonesia Fajar Alfian/Mu...,https://sport.detik.com/raket/d-6978601/harapa...
6,The Daddies dan Pram-Yere Lolos ke Perempat Fi...,Author not found,Category not found,"8,691 Views | Jumat, 13 Okt 2023 04:20 WIB",Dua pasangan ganda putra Indonesia raih kemena...,https://20.detik.com/detikupdate/20231013-2310...
7,"Harris Horatius Sumbang Emas di Asian Games, M...",Randy Prasatya,Sport Lain,"Rabu, 27 Sep 2023 01:31 WIB",Harris Horatius menyumbang medali emas untuk I...,https://sport.detik.com/sport-lain/d-6952381/h...
8,Plot Twist Martin di Mandalika: Start Bak Roke...,Author not found,Category not found,"12,934 Views | Minggu, 15 Okt 2023 16:36 WIB",Jorge Martin start dari grid 6 dan melesat bag...,https://20.detik.com/detikupdate/20231015-2310...
9,Sanggoe Sabet Medali Perak Skateboard Asian Ga...,Mercy Raya,Sport Lain,"Rabu, 27 Sep 2023 17:35 WIB",Atlet skateboard Sanggoe Darma Tanjung raih me...,https://sport.detik.com/sport-lain/d-6953625/s...


Save Data

In [ ]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../tempat_simpan_nasional/detik_sport_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../tempat_simpan_nasional/detik_sport_2023-10-21_11-12-38.xlsx
